In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = "/content/drive/My Drive/Aspect-based Sentiment Analysis/"

In [3]:
!pip install fairseq
!pip3 install vncorenlp
!pip3 install transformers
!pip install fastBPE
!pip install keras-radam

     |████████████████████████████████| 307kB 4.5MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp36-cp36m-linux_x86_64.whl size=2046454 sha256=6798a0ab6a5ce3d624fe33a09fd913cf82c7f965d21efbe0fac24f47682fcd22
  Stored in directory: /root/.cache/pip/wheels/37/3e/1b/0fa30695dcba41e4b0088067fa40f3328d1e8ee78c22cd4766
Successfully built fairseq
     |████████████████████████████████| 2.7MB 4.8MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645934 sha256=8154816022d2f64ab430a55d538b779123dd1ed7f86af3128dafdfaad5c3afa3
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp
     |████████████████████████████████| 1.1MB 5.0MB/s 
     |████████████████████████████████| 3.0MB 29.7MB/s 
     |████████████████████████████████| 1.1MB 42.2MB/s 
     |████████████████████████████████| 890kB 51.7MB/s 
  Created wheel for

In [4]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *

In [5]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2020-10-18 09:15:45--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 99.86.38.27, 99.86.38.111, 99.86.38.8, ...
Connecting to public.vinai.io (public.vinai.io)|99.86.38.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz’

PhoBERT_base_transf 100%[===================>] 307.47M   241MB/s    in 1.3s    

2020-10-18 09:15:47 (241 MB/s) - ‘PhoBERT_base_transformers.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [6]:
phoBERT = 'PhoBERT_base_transformers'

In [7]:
import pickle
import pandas as pd
df = pd.read_csv(path+"VLSP2018/review_train.csv")
df_dev = pd.read_csv(path+'VLSP2018/review_dev.csv')
df_test = pd.read_csv(path+'VLSP2018/review_test.csv')

In [8]:
data_train = df.review.to_list()
label_train = pickle.load(open(path+'VLSP2018/aspect_train.pkl','rb'))
data_dev = df_dev.review.tolist()
label_dev =  pickle.load(open(path+'VLSP2018/aspect_dev.pkl','rb'))
data_test = df_test.review.tolist()
label_test = pickle.load(open(path+'VLSP2018/aspect_test.pkl','rb'))

In [9]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
class BPE():
  bpe_codes = 'PhoBERT_base_transformers/bpe.codes'

args = BPE()
bpe = fastBPE(args)

In [10]:
vocab = Dictionary()
vocab.add_from_file('PhoBERT_base_transformers/dict.txt')

In [11]:
import numpy as np
from tqdm import tqdm
max_sequence_length = 256
def convert_lines(lines, vocab, bpe):
  '''
  lines: list các văn bản input
  vocab: từ điển dùng để encoding subwords
  bpe: 
  '''
  # Khởi tạo ma trận output
  outputs = np.zeros((len(lines), max_sequence_length), dtype=np.int32) # --> shape (number_lines, max_seq_len)
  mask_token = np.zeros((len(lines), max_sequence_length), dtype=np.int32)
  # Index của các token cls (đầu câu), eos (cuối câu), padding (padding token)
  cls_id = 0
  eos_id = 2
  pad_id = 1

  for idx, row in tqdm(enumerate(lines), total=len(lines)): 
    # Mã hóa subwords theo byte pair encoding(bpe)
    subwords = bpe.encode('<s> '+ row +' </s>')
    input_ids = vocab.encode_line(subwords, append_eos=False, add_if_not_exist=False).long().tolist()
    # Truncate input nếu độ dài vượt quá max_seq_len
    if len(input_ids) > max_sequence_length: 
      input_ids = input_ids[:max_sequence_length] 
      input_ids[-1] = eos_id
    else:
      # Padding nếu độ dài câu chưa bằng max_seq_len
      input_ids = input_ids + [pad_id, ]*(max_sequence_length - len(input_ids))
      mask = [0 if i==1 else 1 for i in input_ids]
    outputs[idx,:] = np.array(input_ids)
    mask_token[idx,:] = np.array(mask)
  return outputs, mask_token

In [12]:
X_train, X_train_mask = convert_lines(data_train, vocab, bpe)
X_dev, X_dev_mask = convert_lines(data_dev, vocab, bpe)
X_test, X_test_mask = convert_lines(data_test, vocab, bpe)

100%|██████████| 500/500 [00:00<00:00, 707.20it/s]


In [13]:
y_train = np.array(label_train).astype('float32')
y_dev = np.array(label_dev).astype('float32')
y_test = np.array(label_test).astype('float32')

In [14]:
from tensorflow.keras.layers import Dense, Input, Flatten,SpatialDropout1D,Bidirectional, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, Dropout,GlobalMaxPool1D,Lambda,MaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional,LSTM,GRU
from tensorflow.keras.optimizers import Adam
from keras_radam.training import RAdamOptimizer

In [15]:
config = RobertaConfig.from_pretrained(
  phoBERT+'/config.json',
  output_hidden_states=True,
  num_labels=12
  )
model_bert = TFRobertaModel.from_pretrained(phoBERT+'/model.bin', config=config, from_pt=True)

input_ids = Input(shape=(256,), name='input_token', dtype='int64')
input_mask = Input(shape=(256,), name='mask_token', dtype='int64')

pretrain = model_bert(input_ids, attention_mask=input_mask)[0]
cls_token = pretrain[:,0,:]
dropout_layer = Dropout(0.2)(cls_token)

dense_layer = Dense(768, activation='relu')(dropout_layer)
dense_layer = Dense(512, activation='relu')(dense_layer)
dense_layer = Dense(256, activation='relu')(dense_layer)
preds = Dense(12, activation='sigmoid')(dense_layer)

model = Model([input_ids,input_mask], preds)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [16]:
def f1(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    return 2 * (K.sum(y_true * y_pred)+ K.epsilon()) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())

In [21]:
LR = 2e-5
EPOCHS = 10
BATCH_SIZE = 16

In [18]:
import tensorflow_addons as tfa
optimizer = tfa.optimizers.RectifiedAdam(lr=LR)
loss = tfa.losses.SigmoidFocalCrossEntropy()

In [22]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=LR),
              metrics=[f1])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(path+'phoBERT_checkpoint.h5', monitor='val_f1', verbose=1, save_best_only=True, mode='max')
callback_list = [checkpoint]

In [23]:
model.fit([X_train, X_train_mask], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose = 1, validation_data=([X_test, X_test_mask], y_test))     

Epoch 1/10
186/186 [==============================] - 170s 916ms/step - loss: 0.3794 - f1: 0.5397 - val_loss: 0.5182 - val_f1: 0.6009
Epoch 2/10
186/186 [==============================] - 168s 904ms/step - loss: 0.3104 - f1: 0.6201 - val_loss: 0.5471 - val_f1: 0.5420
Epoch 3/10
186/186 [==============================] - 168s 903ms/step - loss: 0.2668 - f1: 0.6733 - val_loss: 0.4220 - val_f1: 0.6459
Epoch 4/10
186/186 [==============================] - 168s 904ms/step - loss: 0.2288 - f1: 0.7203 - val_loss: 0.3816 - val_f1: 0.7004
Epoch 5/10
186/186 [==============================] - 168s 903ms/step - loss: 0.1968 - f1: 0.7569 - val_loss: 0.3847 - val_f1: 0.7196
Epoch 6/10
186/186 [==============================] - 168s 903ms/step - loss: 0.1789 - f1: 0.7814 - val_loss: 0.3994 - val_f1: 0.7410
Epoch 7/10
186/186 [==============================] - 168s 903ms/step - loss: 0.1532 - f1: 0.8117 - val_loss: 0.3865 - val_f1: 0.7374
Epoch 8/10
186/186 [==============================] - 168s 903

In [24]:
model.fit([X_train, X_train_mask], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose = 1, validation_data=([X_test, X_test_mask], y_test))     

Epoch 1/10
186/186 [==============================] - 168s 904ms/step - loss: 0.0850 - f1: 0.8924 - val_loss: 0.4608 - val_f1: 0.7560
Epoch 2/10
186/186 [==============================] - 168s 903ms/step - loss: 0.0679 - f1: 0.9126 - val_loss: 0.4773 - val_f1: 0.7722
Epoch 3/10
186/186 [==============================] - 168s 904ms/step - loss: 0.0569 - f1: 0.9228 - val_loss: 0.4973 - val_f1: 0.7705
Epoch 4/10
186/186 [==============================] - 168s 904ms/step - loss: 0.0442 - f1: 0.9409 - val_loss: 0.5402 - val_f1: 0.7897
Epoch 5/10
186/186 [==============================] - 168s 903ms/step - loss: 0.0379 - f1: 0.9475 - val_loss: 0.4927 - val_f1: 0.7819
Epoch 6/10
186/186 [==============================] - 168s 904ms/step - loss: 0.0368 - f1: 0.9516 - val_loss: 0.5564 - val_f1: 0.7851
Epoch 7/10
186/186 [==============================] - 168s 903ms/step - loss: 0.0290 - f1: 0.9610 - val_loss: 0.5640 - val_f1: 0.7819
Epoch 8/10
186/186 [==============================] - 168s 903

In [25]:
model.fit([X_train, X_train_mask], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose = 1, validation_data=([X_test, X_test_mask], y_test))     

Epoch 1/10
186/186 [==============================] - 168s 903ms/step - loss: 0.0167 - f1: 0.9772 - val_loss: 0.7220 - val_f1: 0.7782
Epoch 2/10
186/186 [==============================] - 168s 903ms/step - loss: 0.0147 - f1: 0.9792 - val_loss: 0.6777 - val_f1: 0.7862
Epoch 3/10
186/186 [==============================] - 168s 903ms/step - loss: 0.0142 - f1: 0.9804 - val_loss: 0.6972 - val_f1: 0.7885
Epoch 4/10
186/186 [==============================] - 168s 904ms/step - loss: 0.0144 - f1: 0.9803 - val_loss: 0.7071 - val_f1: 0.7990
Epoch 5/10
186/186 [==============================] - 168s 904ms/step - loss: 0.0136 - f1: 0.9820 - val_loss: 0.7848 - val_f1: 0.7957
Epoch 6/10
186/186 [==============================] - 168s 904ms/step - loss: 0.0093 - f1: 0.9866 - val_loss: 0.7924 - val_f1: 0.7941
Epoch 7/10
186/186 [==============================] - 168s 905ms/step - loss: 0.0087 - f1: 0.9877 - val_loss: 0.7746 - val_f1: 0.7970
Epoch 8/10
186/186 [==============================] - 168s 904

In [ ]:
model.fit([X_train, X_train_mask], y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose = 1, validation_data=([X_test, X_test_mask], y_test))     

Epoch 1/10
186/186 [==============================] - 168s 903ms/step - loss: 0.0084 - f1: 0.9889 - val_loss: 0.8157 - val_f1: 0.8153
Epoch 2/10
186/186 [==============================] - 168s 903ms/step - loss: 0.0059 - f1: 0.9915 - val_loss: 0.8966 - val_f1: 0.7959
Epoch 3/10
186/186 [==============================] - 168s 902ms/step - loss: 0.0049 - f1: 0.9928 - val_loss: 0.9048 - val_f1: 0.7907
Epoch 4/10
 30/186 [===>..........................] - ETA: 2:10 - loss: 0.0075 - f1: 0.9904

In [ ]:
model.save_weights(path+'phobert_checkpoint.h5')

In [ ]:
model.load_weights(path+'phobert_checkpoint.h5')